In [2]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.6 MB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 61.0 MB 5.8 MB/s eta 0:00:01
     |████████████████████████████████| 176 kB 7.3 MB/s eta 0:00:01
     |████████████████████████████████| 5.7 MB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 536 kB 6.3 MB/s eta 0:00:01
  ERROR: Command errored out with exit status 1:
   command: /Library/Developer/CommandLineTools/usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/mz/2zw1gjvs1p93ftdnzz9nfkz80000gn/T/pip-install-g499o6pm/pytorch_97c0da1e061d4e6f89ae75479a1b2a73/setup.py'"'"'; __file__='"'"'/private/var/folders/mz/2zw1gjvs1p93ftdnzz9nfkz80000gn/T/pip-install-g499o6pm/pytorch_97c0da1e061d4e6f89ae75479a1b2a73/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools

In [3]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import CelebA # Training dataset
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0) # Set for testing purposes, please do not change!  

/Users/snehasundar/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def show_tensor_images(image_tensor, num_images=25, size=(3, 64, 64)):
    '''
    Function for visualizing images: Given a tensor of images, number of images, and
    size per image, plots and prints the images in a uniform grid.
    '''
    image_unflat = image_tensor.detach().cpu().view(-1, *size)
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.axis('off')  # Hide axes for better visualization
    plt.show()

In [5]:
def get_generator_block(input_dim, output_dim):
    '''
    Function for returning a block of the generator's neural network
    given input and output dimensions.
    Parameters:
        input_dim: the dimension of the input vector, a scalar
        output_dim: the dimension of the output vector, a scalar
    Returns:
        a generator neural network layer, with a linear transformation 
          followed by a batch normalization and then a relu activation
    '''
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace=True)
    )

In [12]:
# build the generator class
class Generator(nn.Module):
    
    def __init__(self, z_dim=100, im_dim=12288, hidden_dim=64):
        super(Generator, self).__init__()
        # build the neural network
        self.gen = nn.Sequential(
            get_generator_block(z_dim, hidden_dim * 4),
            get_generator_block(hidden_dim * 4, hidden_dim * 8),
            get_generator_block(hidden_dim * 8, hidden_dim * 16),
            nn.Linear(hidden_dim * 16, im_dim),
            nn.Tanh()  # Use Tanh activation for the last layer to output values in [-1, 1]
        )
    
    def forward(self, noise):
        return self.gen(noise)

In [23]:
def get_noise(batch_size, z_dim, device='cpu'):
    '''
    Function to generate a batch of noise vectors (z) that will be used as input to the generator.
    
    Parameters:
        batch_size: Size of the batch, a scalar
        z_dim: Dimension of the noise vector (z), a scalar
        device: Device (CPU or GPU) where the tensor will be allocated
    
    Returns:
        A tensor of shape (batch_size, z_dim) containing noise vectors sampled from a normal distribution
    '''
    return torch.randn(batch_size, z_dim, device=device)

In [24]:
def get_discriminator_block(input_dim, output_dim):
    '''
    Discriminator Block
    Function for returning a neural network block of the discriminator given input and output dimensions.
    
    Parameters:
        input_dim: the dimension of the input vector, a scalar
        output_dim: the dimension of the output vector, a scalar
        
    Returns:
        a discriminator neural network layer, with a linear transformation 
        followed by an nn.LeakyReLU activation with negative slope of 0.2
    '''
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.LeakyReLU(0.2, inplace=True)
    )

In [ ]:
# build the discriminator class
class Discriminator(nn.Module):
    def __init__(self, im_dim=12288, hidden_dim=64):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            get_discriminator_block(im_dim, hidden_dim * 4),
            get_discriminator_block(hidden_dim * 4, hidden_dim * 2),
            get_discriminator_block(hidden_dim * 2, hidden_dim),
            nn.Linear(hidden_dim, 1)
        )
